# Level 2 
## Предлагается сначала просмотреть весь ноутбук и только потом запускать ячейки, поскольку последний блок с вычислением средней награды может быть запущен с использованием уже сохраненных на гите моделей, то есть без необходимости повторно запускать обучение.

#### Запуск и работоспособность были проверены в Google Colab на GPU T4. Ожидается, что все требуемые библиотеки уже установлены в окружении из ноутбука Level1.ipynb , но на всякий случай продублируем здесь.

### Блок установки библиотек. 
(Полный дубликат из Level1, хотя trl в этом ноутбуке не используется) 

In [ ]:
!pip install --upgrade \
    diffusers==0.19.3 \
    accelerate==0.23.0 \
    sentence-transformers==2.2.2

In [ ]:
!pip install --upgrade huggingface_hub==0.25.0

In [ ]:
pip install fsspec==2023.6.0 gcsfs==2023.6.0

In [ ]:
!pip install --upgrade "jax[cuda]==0.4.27" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [ ]:
pip install transformers datasets trl==0.14.0 peft==0.5.0


Напомним, что в коде все пути к сохраненным моделям указаны в формате, в котором они предполагаются на Google Colab. При этом предлагается заменить "  #YOUR PATH HERE " на тот путь, в которой были установлены сохраненные модели, скачанные с гита. В случае работы в Google Colab они при загрузке автоматически попадают в путь "/content/..." 

Загружаем датасет и делим на train/validation

In [2]:
from datasets import load_dataset

data = load_dataset("esfrankel17/HelpSteer2_binarized")
ds = data["average_rating_split"]

split_data = ds.train_test_split(test_size=0.1, seed=42)
train_dataset = split_data["train"]
validation_dataset = split_data["test"]

print("Train size:", len(train_dataset))
print("Validation size:", len(validation_dataset))
print(train_dataset.features)

Train size: 7810
Validation size: 868
{'prompt': Value(dtype='string', id=None), 'chosen': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}], 'chosen_rating': Value(dtype='float64', id=None), 'rejected': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}], 'rejected_rating': Value(dtype='float64', id=None)}


### Блок с подготовкой RM, которая выдаёт не скалярную оценку, а распределение вероятности поверх дискретных оценок. 

In [1]:
# =============================================================================
# 1. Настройки и гиперпараметры (совпадают с Level 1)
# =============================================================================

import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 1         # подберите в соответствии с ресурсами
learning_rate = 5e-5
num_epochs = 1         # можно увеличить число эпох
max_length = 64       # максимальная длина токенизированного текста

In [ ]:
# =============================================================================
# 2. Функции для преобразования сообщений
# =============================================================================
def merge_messages(messages):
    """
    Преобразуем список словарей [{"role": ..., "content": ...}, ...]
    в строку вида:
        "User: ...\nAssistant: ...\n..."
    """
    text_list = []
    for m in messages:
        role = m.get("role", "")
        content = m.get("content", "")
        text_list.append(f"{role.capitalize()}: {content}")
    return "\n".join(text_list)

def process_item(item):
    """
    Если item уже строка — возвращаем её,
    если это список (например, список сообщений) — объединяем с помощью merge_messages,
    иначе приводим к строке.
    """
    if isinstance(item, str):
        return item
    elif isinstance(item, list):
        return merge_messages(item)
    else:
        return str(item)

In [ ]:
# =============================================================================
# 3. Загрузка модели Reward Model с 10 выходными логитами
# =============================================================================

reward_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"  # имя или путь к модели
# Загружаем модель для классификации с 10 классами (оценки 1..10)
model_rm = AutoModelForSequenceClassification.from_pretrained(
    reward_model_name,
    num_labels=10
)
model_rm.to(device)
model_rm.train()

# Токенизатор для модели RM
tokenizer_rm = AutoTokenizer.from_pretrained(reward_model_name)
if tokenizer_rm.pad_token is None:
    tokenizer_rm.pad_token = tokenizer_rm.eos_token

In [ ]:
# =============================================================================
# 4. Подготовка DataLoader
# =============================================================================

def collate_fn(batch):
    """
    Каждый элемент batch — словарь с ключами, например, "prompt", "chosen", "rejected", ...
    Для ключей "chosen" и "rejected" применяем process_item для преобразования в строку.
    """
    collated = {}
    for key in batch[0].keys():
        if key in ["chosen", "rejected"]:
            collated[key] = [process_item(sample[key]) for sample in batch]
        else:
            collated[key] = [sample[key] for sample in batch]
    return collated

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

optimizer = torch.optim.AdamW(model_rm.parameters(), lr=learning_rate)

In [ ]:
# =============================================================================
# 5. Обучающий цикл с функцией потерь
# =============================================================================

# Подготавливаем маску для суммирования: создаем матрицу 10x10, где mask[i, j] = 1, если i > j,
# что соответствует суммированию p_i(chosen)*p_j(rejected) для всех i > j.
mask = torch.tril(torch.ones(10, 10), diagonal=-1).to(device)  # размер: (10,10)

print("Начало обучения Reward Model с распределением оценок...")
for epoch in range(num_epochs):
    for step, batch in enumerate(train_loader):
        # Извлекаем списки текстов для выбранного и отвергнутого ответов
        chosen_texts = batch["chosen"]
        rejected_texts = batch["rejected"]

        # Токенизируем выбранные и отвергнутые тексты
        chosen_encodings = tokenizer_rm(
            chosen_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        )
        rejected_encodings = tokenizer_rm(
            rejected_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        )
        # Переносим на устройство
        chosen_encodings = {k: v.to(device) for k, v in chosen_encodings.items()}
        rejected_encodings = {k: v.to(device) for k, v in rejected_encodings.items()}

        # Прямой проход: получаем логиты для выбранного и отвергнутого ответов
        outputs_chosen = model_rm(**chosen_encodings)    # shape: (batch_size, 10)
        outputs_rejected = model_rm(**rejected_encodings)  # shape: (batch_size, 10)

        logits_chosen = outputs_chosen.logits
        logits_rejected = outputs_rejected.logits

        # Преобразуем логиты в распределения вероятностей по классам
        p_chosen = torch.softmax(logits_chosen, dim=-1)    # shape: (batch_size, 10)
        p_rejected = torch.softmax(logits_rejected, dim=-1)  # shape: (batch_size, 10)

        # Вычисляем вероятность того, что оценка выбранного ответа выше, чем отвергнутого:
        # p_win = sum_{i=0}^{9} sum_{j=0}^{i-1} p_chosen[i] * p_rejected[j]
        # Реализуем это в векторизованном виде:
        p_chosen_unsq = p_chosen.unsqueeze(2)    # (batch_size, 10, 1)
        p_rejected_unsq = p_rejected.unsqueeze(1)  # (batch_size, 1, 10)
        prod = p_chosen_unsq * p_rejected_unsq      # (batch_size, 10, 10)
        p_win = (prod * mask).view(prod.size(0), -1).sum(dim=1)  # (batch_size,)

        # Чтобы избежать log(0), делаем clamp
        p_win = p_win.clamp(min=1e-8)

        # Функция потерь: усредненное по батчу значение -log(p_win)
        loss = - torch.log(p_win).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 10 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")

print("Обучение Reward Model завершено.")


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Начало обучения Reward Model с распределением оценок...
Epoch 1, Step 0, Loss: 1.1581
Epoch 1, Step 10, Loss: 1.9186
Epoch 1, Step 20, Loss: 1.1511
Epoch 1, Step 30, Loss: 0.9796
Epoch 1, Step 40, Loss: 1.6202
Epoch 1, Step 50, Loss: 1.1592
Epoch 1, Step 60, Loss: 1.1645
Epoch 1, Step 70, Loss: 1.1340
Epoch 1, Step 80, Loss: 1.2958
Epoch 1, Step 90, Loss: 0.9211
Epoch 1, Step 100, Loss: 0.1259
Epoch 1, Step 110, Loss: 1.2483
Epoch 1, Step 120, Loss: 0.9974
Epoch 1, Step 130, Loss: 0.5602
Epoch 1, Step 140, Loss: 0.5220
Epoch 1, Step 150, Loss: 0.3325
Epoch 1, Step 160, Loss: 0.9945
Epoch 1, Step 170, Loss: 0.4272
Epoch 1, Step 180, Loss: 0.5015
Epoch 1, Step 190, Loss: 1.4736
Epoch 1, Step 200, Loss: 0.8810
Epoch 1, Step 210, Loss: 0.9783
Epoch 1, Step 220, Loss: 1.0180
Epoch 1, Step 230, Loss: 0.8734
Epoch 1, Step 240, Loss: 0.7269
Epoch 1, Step 250, Loss: 1.1481
Epoch 1, Step 260, Loss: 0.9211
Epoch 1, Step 280, Loss: 0.8411
Epoch 1, Step 290, Loss: 0.8735
Epoch 1, Step 300, Loss: 0.

In [ ]:
model_rm.save_pretrained("./rm-prob-checkpoints")
tokenizer_rm.save_pretrained("./rm-prob-checkpoints")
# Помним, что в Google Colab они сохраняются по факту в "/content/rm-prob-checkpoints" 

('./rm-prob-checkpoints/tokenizer_config.json',
 './rm-prob-checkpoints/special_tokens_map.json',
 './rm-prob-checkpoints/vocab.json',
 './rm-prob-checkpoints/merges.txt',
 './rm-prob-checkpoints/added_tokens.json',
 './rm-prob-checkpoints/tokenizer.json')

## REINFORCE поверх SFT с вероятностной RM
Для запуска этого блока нужно запустить только блок с датасетом, потому что RM мы уже сохранили.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# =============================================================================
# 0. Загрузка моделей и токенизаторов
# =============================================================================
# sft_model, tokenizer – SFT модель, которую мы будем дообучать с помощью REINFORCE
sft_model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct", padding_side="left")
sft_model.to(device)

# Загрузим вероятностную RM, обученную ранее (с 10 выходными логитами)
# Предполагаем, что модель уже обучена и находится в папке "rm-prob-checkpoints"
rm_model_path = #YOUR PATH HERE
# в случае запуска в Google Colab: "/content/rm-prob-checkpoints" 
model_rm = AutoModelForSequenceClassification.from_pretrained(rm_model_path, num_labels=10)
model_rm.to(device)
model_rm.eval()  # Для оценки награды – не обновляем RM

# Токенизатор для RM
tokenizer_rm = AutoTokenizer.from_pretrained(rm_model_path)
if tokenizer_rm.pad_token is None:
    tokenizer_rm.pad_token = tokenizer_rm.eos_token

In [ ]:
# =============================================================================
# 1. Гиперпараметры и оптимизатор (совпадают с Level 1)
# =============================================================================

batch_size = 4   
num_iterations = 30
max_new_tokens = 50       # максимальное число генерируемых токенов (отвечает за длину ответа)
alpha = 0.9               # коэффициент для moving average baseline
learning_rate = 1e-5      # скорость обучения для SFT-модели


optimizer = torch.optim.AdamW(sft_model.parameters(), lr=learning_rate)

In [ ]:
# =============================================================================
# 2. Подготовка DataLoader для датасета
# =============================================================================

# Используем DataLoader, который выдаёт батчи в виде словаря.
from torch.utils.data import DataLoader
import itertools

dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
data_iter = iter(itertools.cycle(dataloader))  # чтобы можно было бесконечно брать батчи

In [ ]:
# =============================================================================
# 3. Вспомогательные функции
# =============================================================================

def compute_log_prob(prompt, generated_ids):
    """
    Вычисляет суммарную log‑вероятность сгенерированных токенов (исключая prompt).
    Для этого:
      - Токенизируем prompt, чтобы узнать его длину.
      - Пропускаем всю последовательность (prompt + сгенерированный ответ) через модель в режиме teacher forcing.
      - Считаем log‑вероятность сгенерированных токенов (начиная с позиции len(prompt)).
    """
    prompt_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    prompt_length = prompt_ids.size(1)

    full_ids = generated_ids  # shape: [1, T]
    T = full_ids.size(1)
    if T - prompt_length <= 0:
        return torch.tensor(0.0, device=device)

    outputs = sft_model(full_ids)
    logits = outputs.logits  # shape: [1, T, vocab_size]
    log_probs = torch.log_softmax(logits, dim=-1)

    # Сдвигаем последовательность: для токенов с позиции prompt_length ... T-1
    pred_logits = log_probs[:, prompt_length - 1:T - 1, :]
    target_tokens = full_ids[:, prompt_length:]
    token_log_probs = pred_logits.gather(2, target_tokens.unsqueeze(-1)).squeeze(-1)
    sum_log_prob = token_log_probs.sum()
    return sum_log_prob

def compute_expected_reward(text):
    """
    Вычисляет ожидаемое значение оценки, используя вероятностную RM.
    Токенизируем текст, пропускаем через RM, применяем softmax и вычисляем:
      reward = sum_{i=1}^{10} (i * p_i)
    Дополнительно можно вычислить энтропию или дисперсию, если требуется.
    """
    with torch.no_grad():
        tokenized = tokenizer_rm(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(device)
        outputs = model_rm(**tokenized)  # shape: [1, 10]
        logits = outputs.logits  # [1, 10]
        probs = torch.softmax(logits, dim=-1)  # [1, 10]
        # Оценки от 1 до 10
        rating_values = torch.arange(1, 11, device=device).float()  # [10]
        expected_reward = (probs * rating_values).sum()  # скаляр
    return expected_reward

In [ ]:
# =============================================================================
# 4. Основной цикл обучения REINFORCE с вероятностной RM
# =============================================================================

# Инициализируем baseline (moving average) – начнём с 0
baseline = 0.0

print("Начало обучения REINFORCE с вероятностной RM...")
for step in range(num_iterations):
    batch = next(data_iter)  # batch — словарь с ключом "prompt"

    batch_loss = 0.0
    batch_rewards = []  # для обновления baseline

    # Обрабатываем каждый пример в батче по индексам
    for i in range(len(batch["prompt"])):
        prompt_text = batch["prompt"][i]

        # 1. Генерация ответа SFT-моделью
        encoded_prompt = tokenizer(prompt_text, return_tensors="pt", truncation=True).to(device)
        generated_ids = sft_model.generate(
            input_ids=encoded_prompt.input_ids,
            attention_mask=encoded_prompt.attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id
        )
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # 2. Вычисляем log‑вероятность сгенерированного ответа
        log_prob = compute_log_prob(prompt_text, generated_ids)

        # 3. Вычисляем награду как ожидаемое значение оценки, используя RM
        reward_value = compute_expected_reward(generated_text)
        batch_rewards.append(reward_value.item())

        # 4. Вычисляем Advantage = (reward - baseline)
        advantage = reward_value - baseline

        # 5. Функция потерь REINFORCE: - advantage * log_prob
        sample_loss = - advantage * log_prob
        batch_loss = batch_loss + sample_loss

    # Усредняем потери по батчу
    batch_loss = batch_loss / batch_size

    # Обновляем параметры SFT-модели
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

    # Обновляем baseline как скользящее среднее наград
    batch_mean_reward = sum(batch_rewards) / len(batch_rewards)
    baseline = alpha * baseline + (1 - alpha) * batch_mean_reward

    if (step + 1) % 10 == 0:
        print(f"Step {step+1}/{num_iterations} | Loss: {batch_loss.item():.4f} | "
              f"Avg Reward: {batch_mean_reward:.4f} | Baseline: {baseline:.4f}")

print("Обучение REINFORCE завершено.")

# =============================================================================
# 5. Сохранение новой модели 
# =============================================================================

save_path = "./sft-rlhf-prob-model"
sft_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Новая модель сохранена в {save_path}")


Начало обучения REINFORCE с вероятностной RM...
Step 10/30 | Loss: 180.3823 | Avg Reward: 5.4709 | Baseline: 3.4580
Step 20/30 | Loss: 61.6600 | Avg Reward: 5.7487 | Baseline: 4.8598
Step 30/30 | Loss: 10.1747 | Avg Reward: 4.8728 | Baseline: 5.3448
Обучение REINFORCE завершено.
Новая модель сохранена в ./sft-rlhf-prob-model


### Оценим среднюю награду модели на validation наборе
Эти ячейки можно запускать независимо от верхних блоков, но при условии запуска ячеек с датасетом. Без запуска блока датасета работать не будет. Здесь используются модели, которые мы уже получили(сохранили) в верхних блоках.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# путь, где сохранена RM
rm_model_path = #YOUR PATH HERE
# в случае запуска в Google Colab: "/content/rm-prob-checkpoints" 
model_rm = AutoModelForSequenceClassification.from_pretrained(rm_model_path, num_labels=10)
model_rm.to(device)
model_rm.eval()  # Для оценки награды – не обновляем RM

sft_model = AutoModelForCausalLM.from_pretrained(#YOUR PATH HERE
# в случае запуска в Google Colab: "/content/sft-rlhf-prob-model"
    )
tokenizer = AutoTokenizer.from_pretrained(#YOUR PATH HERE
# в случае запуска в Google Colab: "/content/sft-rlhf-prob-model"
                                          , padding_side="left")
sft_model.to(device)


# Функция для оценки средней награды модели на validation наборе
def evaluate_avg_reward(model, tokenizer, dataset, reward_model, reward_tokenizer, max_new_tokens=50):
    model.eval()
    rewards = []
    # Пройдемся по всем примерам в validation наборе (можно ограничить число примеров для ускорения)
    for sample in dataset:
        prompt_text = sample["prompt"]
        encoded = tokenizer(prompt_text, return_tensors="pt", truncation=True).to(device)
        generated_ids = model.generate(
            input_ids=encoded.input_ids,
            attention_mask=encoded.attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id
        )
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        with torch.no_grad():
            tokenized_reward = reward_tokenizer(
                generated_text,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(device)
            reward_output = reward_model(**tokenized_reward)
            # Предполагаем, что reward_model возвращает логиты (скаляр)
            reward_value = reward_output.logits.mean().item()
        rewards.append(reward_value)
    avg_reward = sum(rewards) / len(rewards)
    return avg_reward

# -------------------------------
# RLHF модель – текущая sft_model после дообучения REINFORCE
# -------------------------------

avg_reward_rlhf = evaluate_avg_reward(
    model=sft_model,
    tokenizer=tokenizer,
    dataset=validation_dataset,
    reward_model=model_rm,
    reward_tokenizer=tokenizer_rm,
    max_new_tokens=50
)
print("Средняя награда RLHF модели с вероятностной RM:", avg_reward_rlhf)


Средняя награда RLHF модели с вероятностной RM: 0.33886930197133996


Обуждение результатов хранится в виде отчета на гите.